ICI ca test de recup des infos sur nba.com

import all the libraries

In [1]:
from selenium import webdriver #/!\ version :4.5.0
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options


from bs4 import BeautifulSoup

import pandas as pd

import time

Constants

In [8]:
#create a constant for the url
NBA_PLAYERS_STATS_URL = 'https://www.nba.com/stats/alltime-leaders'
NBA_PLAYERS_INFO_URL = 'https://www.nba.com/players'

# path to the file containing the chrome driver
CHROME_DRIVER_PATH = 'C:/chromedriver.exe'

Useful functions for browser

In [3]:
# Starting the browser.
def startBrowser():
    s=Service(CHROME_DRIVER_PATH)
    options = webdriver.ChromeOptions()
    options = Options()
    options.add_argument("--headless")
    return  webdriver.Chrome(service=s, options=options)

# Checking if the browser is closed.
def isBrowserClosed(browser):
    isbrowserClosed = False
    try:
        webdriver.title
    except:
        isbrowserClosed = True
    return isbrowserClosed
        

Get stats from all players

In [10]:
# The above code is getting the data from the URL and returning the dataframe.
def getDataFromURL(URL):
    browser = startBrowser()
    #open the url
    browser.get(URL)

    # accept cookies
    browser.find_element(By.CSS_SELECTOR, "button#onetrust-accept-btn-handler").click()
    time.sleep(10)

    df = pd.DataFrame()

    while browser.find_element(By.CSS_SELECTOR, "button[title^='Next Page Button']").is_enabled():
        html = browser.page_source
        data, headers = getDataFromHTML(html)

        df = pd.concat([df, pd.DataFrame(data, columns=headers)], ignore_index=True)

        print(".", end = '')

        while True:
            try:
                browser.find_element(By.CSS_SELECTOR, "button[title^='Next Page Button']").click()
                break
            except:
                if(isBrowserClosed(browser)):
                    return df
                else:
                    print('not yet clickable')
                    continue

        html = browser.page_source
        data, headers = getDataFromHTML(html)

        df = pd.concat([df, pd.DataFrame(data, columns=headers)], ignore_index=True)

        print(".", end = '')

    #close the browser
    browser.quit()
    #return the dataframe
    return df



#Parsing the html and returning the data and headers.
def getDataFromHTML(html):
    #parse the html
    soup = BeautifulSoup(html, 'html.parser')
    #find the table with class Crom_table__p1iZz or players-list
    table = soup.find('table', attrs = {'class' : ['Crom_table__p1iZz','players-list']})
    #get the table headers
    headers = [header.text for header in table.findAll('th', attrs = {'hidden': None})]
    #get the table rows
    rows = table.find_all('tr')
    #get the table data
    data = [[td.text for td in rows[i].find_all('td')] for i in range(len(rows))]
    data = [row for row in data if row != []]#they is an empty at the start idk why but yes

    return data, headers


Save stats in a csv file

In [9]:
print("start of the scrapping")
df = getDataFromURL(NBA_PLAYERS_STATS_URL)
print("\nend of the scrapping\n\n")



print("saving the dataframe to a csv file...")
df.to_csv('nbaPlayersAllTimesData.csv', index = False)
print("done")

start of the scrapping
..............................................................................................................................................................................................
end of the scrapping


saving the dataframe to a csv file...
done


In [17]:
print("start of the scrapping")
df = getDataFromURL(NBA_PLAYERS_INFO_URL)
print("\nend of the scrapping\n\n")

print("Converting height and weight to cm and kg...", end = '')
#convert height in df to cm
def convertHeightToCm(height):
    feet = int(height.split('-')[0])
    inches = int(height.split('-')[1])
    return (feet * 12 + inches) * 2.54

#convert weight in df to kg
def convertWeightToKg(weight):
    weight = weight[:-3]
    return int(weight) * 0.453592

#convert height in df to cm
df['Height'] = df['Height'].apply(convertHeightToCm)
# convert weight in df to kg
df['Weight'] = df['Weight'].apply(convertWeightToKg)
print("done\n\n")

print("saving the dataframe to a csv file...", end = '')
df.to_csv('nbaPlayersInfo.csv', index = False)
print("done")

start of the scrapping
....................
end of the scrapping


Converting height and weight to cm and kg...
done


saving the dataframe to a csv file...
done


            Player Team Number Position Height   Weight    Last Attended  \
0  PreciousAchiuwa  TOR      5        F    6-8  225 lbs          Memphis   
1      StevenAdams  MEM      4        C   6-11  265 lbs       Pittsburgh   
2       BamAdebayo  MIA     13      C-F    6-9  255 lbs         Kentucky   
3      OchaiAgbaji  UTA     30        G    6-5  215 lbs           Kansas   
4      SantiAldama  MEM      7      F-C    7-0  215 lbs  Loyola-Maryland   

       Country  
0      Nigeria  
1  New Zealand  
2          USA  
3          USA  
4        Spain  
            Player Team Number Position  Height     Weight    Last Attended  \
0  PreciousAchiuwa  TOR      5        F  203.20  102.05820          Memphis   
1      StevenAdams  MEM      4        C  210.82  120.20188       Pittsburgh   
2       BamAdebayo  MIA     13      C-F  205.74  115.66596         Kentucky   
3      OchaiAgbaji  UTA     30        G  195.58   97.52228           Kansas   
4      SantiAldama  MEM      7      F-C  213.3